# Exercice 1

## Import

In [1]:
#Import
import pyspark
import findspark
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import SQLContext
import pyspark.sql.functions as F
from pyspark.sql import DataFrameReader

In [2]:
#Initialisation de findspark
findspark.init()

In [3]:
#Création du SparkSession
spark = SparkSession.builder.appName("Exemple Spark").getOrCreate()
sc = SparkContext.getOrCreate()

## Create a dataframe

In [4]:
lstDict = [{'prenom': 'Kareem','nom': 'Abdul-jabbar', 'points': 1560},
           {'prenom': 'Karl','nom': 'Malone','points': 1476},
           {'prenom': 'Michael','nom': 'Jordan','points': 1072},
           {'prenom': 'Wilt','nom': 'Chamberlain','points': 1045},
           {'prenom': 'Oneil','nom': 'Shaquille','points': 1147}]

In [5]:
rdd = sc.parallelize(lstDict)

In [6]:
sqlContext = SQLContext(sc)
df = sqlContext.createDataFrame(rdd)
df.show()

+------------+------+-------+
|         nom|points| prenom|
+------------+------+-------+
|Abdul-jabbar|  1560| Kareem|
|      Malone|  1476|   Karl|
|      Jordan|  1072|Michael|
| Chamberlain|  1045|   Wilt|
|   Shaquille|  1147|  Oneil|
+------------+------+-------+



## Rajouter une colonne

Créer une fonction udf pour ajouter une colonne lensup10 avec true si le nom de la personne dépasse 10 caractères sinon false

In [7]:
udf_len_sup_10 = F.udf(lambda x : len(x) >= 10, BooleanType())

In [8]:
df.withColumn('len_sup_10', udf_len_sup_10(df.nom)).show()

+------------+------+-------+----------+
|         nom|points| prenom|len_sup_10|
+------------+------+-------+----------+
|Abdul-jabbar|  1560| Kareem|      true|
|      Malone|  1476|   Karl|     false|
|      Jordan|  1072|Michael|     false|
| Chamberlain|  1045|   Wilt|      true|
|   Shaquille|  1147|  Oneil|     false|
+------------+------+-------+----------+



# Exercice 2

## Méthode 1 pour lire un dataframe depuis un csv

In [9]:
df = (sqlContext.read.format("com.databricks.spark.csv").option("header","true").load("./Exercices_Dataframe/fakefriends_header.csv"))
df.show()

+------+--------+---+-------+
|userID|    name|age|friends|
+------+--------+---+-------+
|     0|    Will| 33|    385|
|     1|Jean-Luc| 26|      2|
|     2|    Hugh| 55|    221|
|     3|  Deanna| 40|    465|
|     4|   Quark| 68|     21|
|     5|  Weyoun| 59|    318|
|     6|  Gowron| 37|    220|
|     7|    Will| 54|    307|
|     8|  Jadzia| 38|    380|
|     9|    Hugh| 27|    181|
|    10|     Odo| 53|    191|
|    11|     Ben| 57|    372|
|    12|   Keiko| 54|    253|
|    13|Jean-Luc| 56|    444|
|    14|    Hugh| 43|     49|
|    15|     Rom| 36|     49|
|    16|  Weyoun| 22|    323|
|    17|     Odo| 35|     13|
|    18|Jean-Luc| 45|    455|
|    19|  Geordi| 60|    246|
+------+--------+---+-------+
only showing top 20 rows



## Méthode 2 pour lire un dataframe depuis un csv

Méthode la plus pratique, a utiliser

In [10]:
df = (spark.read.format("csv").option("header","true").load("./Exercices_Dataframe/fakefriends_header.csv"))
df.show()

+------+--------+---+-------+
|userID|    name|age|friends|
+------+--------+---+-------+
|     0|    Will| 33|    385|
|     1|Jean-Luc| 26|      2|
|     2|    Hugh| 55|    221|
|     3|  Deanna| 40|    465|
|     4|   Quark| 68|     21|
|     5|  Weyoun| 59|    318|
|     6|  Gowron| 37|    220|
|     7|    Will| 54|    307|
|     8|  Jadzia| 38|    380|
|     9|    Hugh| 27|    181|
|    10|     Odo| 53|    191|
|    11|     Ben| 57|    372|
|    12|   Keiko| 54|    253|
|    13|Jean-Luc| 56|    444|
|    14|    Hugh| 43|     49|
|    15|     Rom| 36|     49|
|    16|  Weyoun| 22|    323|
|    17|     Odo| 35|     13|
|    18|Jean-Luc| 45|    455|
|    19|  Geordi| 60|    246|
+------+--------+---+-------+
only showing top 20 rows



## Déterminer le nombre moyen d’amis ventilé par âge des personnes dans ce réseau social.

In [11]:
df2 = df.selectExpr("cast(userID as int) userID",
    "cast(name as string) name",
    "cast(age as int) age",
    "cast(friends as int) friends")
df2.printSchema()
df2.show()

root
 |-- userID: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- friends: integer (nullable = true)

+------+--------+---+-------+
|userID|    name|age|friends|
+------+--------+---+-------+
|     0|    Will| 33|    385|
|     1|Jean-Luc| 26|      2|
|     2|    Hugh| 55|    221|
|     3|  Deanna| 40|    465|
|     4|   Quark| 68|     21|
|     5|  Weyoun| 59|    318|
|     6|  Gowron| 37|    220|
|     7|    Will| 54|    307|
|     8|  Jadzia| 38|    380|
|     9|    Hugh| 27|    181|
|    10|     Odo| 53|    191|
|    11|     Ben| 57|    372|
|    12|   Keiko| 54|    253|
|    13|Jean-Luc| 56|    444|
|    14|    Hugh| 43|     49|
|    15|     Rom| 36|     49|
|    16|  Weyoun| 22|    323|
|    17|     Odo| 35|     13|
|    18|Jean-Luc| 45|    455|
|    19|  Geordi| 60|    246|
+------+--------+---+-------+
only showing top 20 rows



In [125]:
df2.groupBy("age").mean("friends").orderBy("age").show()

+---+------------------+
|age|      avg(friends)|
+---+------------------+
| 18|           343.375|
| 19|213.27272727272728|
| 20|             165.0|
| 21|           350.875|
| 22|206.42857142857142|
| 23|             246.3|
| 24|             233.8|
| 25|197.45454545454547|
| 26|242.05882352941177|
| 27|           228.125|
| 28|             209.1|
| 29|215.91666666666666|
| 30| 235.8181818181818|
| 31|            267.25|
| 32| 207.9090909090909|
| 33| 325.3333333333333|
| 34|             245.5|
| 35|           211.625|
| 36|             246.6|
| 37|249.33333333333334|
+---+------------------+
only showing top 20 rows



# Exercice 3

In [126]:
schema = StructType([\
    StructField("station", StringType(), True),\
    StructField("id", IntegerType(), True),\
    StructField("Temp", StringType(), True),\
    StructField("Value", IntegerType(), True),\
    StructField("col4", StringType(), True),\
    StructField("col5", StringType(), True),\
    StructField("col6", StringType(), True),\
    StructField("col7", StringType(), True)])

In [127]:
df = (spark.read.format("csv").option("header","false").schema(schema).load("./Exercices_Dataframe/1800.csv"))
df.printSchema()
df.show()

root
 |-- station: string (nullable = true)
 |-- id: integer (nullable = true)
 |-- Temp: string (nullable = true)
 |-- Value: integer (nullable = true)
 |-- col4: string (nullable = true)
 |-- col5: string (nullable = true)
 |-- col6: string (nullable = true)
 |-- col7: string (nullable = true)

+-----------+--------+----+-----+----+----+----+----+
|    station|      id|Temp|Value|col4|col5|col6|col7|
+-----------+--------+----+-----+----+----+----+----+
|ITE00100554|18000101|TMAX|  -75|null|null|   E|null|
|ITE00100554|18000101|TMIN| -148|null|null|   E|null|
|GM000010962|18000101|PRCP|    0|null|null|   E|null|
|EZE00100082|18000101|TMAX|  -86|null|null|   E|null|
|EZE00100082|18000101|TMIN| -135|null|null|   E|null|
|ITE00100554|18000102|TMAX|  -60|null|   I|   E|null|
|ITE00100554|18000102|TMIN| -125|null|null|   E|null|
|GM000010962|18000102|PRCP|    0|null|null|   E|null|
|EZE00100082|18000102|TMAX|  -44|null|null|   E|null|
|EZE00100082|18000102|TMIN| -130|null|null|   E|null|


## Afficher la température minimum et sa station associée.

In [129]:
df_tmin = df.filter(df.Temp == "TMIN")

In [130]:
df_tmin.groupBy("station").min("Value").show()

+-----------+----------+
|    station|min(Value)|
+-----------+----------+
|ITE00100554|      -148|
|EZE00100082|      -135|
+-----------+----------+



In [131]:
df_tmin.select([min("Value")]).show()

+----------+
|min(Value)|
+----------+
|      -148|
+----------+



In [134]:
df_tmin.agg({'Value': 'min'}).show()

+----------+
|min(Value)|
+----------+
|      -148|
+----------+



# Exercice 4

## A partir du fichier Lorem_Ipsum.txt. Compter les occurrences de mots à l’aide des méthodes issues des dataframes.

In [18]:
dflines=spark.read.text("./Exercices_Dataframe/Lorem_Ipsum.txt")

In [21]:
dflines.show()

+--------------------+
|               value|
+--------------------+
|What is Lorem Ipsum?|
|Lorem Ipsum is si...|
|                    |
|   Why do we use it?|
|It is a long esta...|
|                    |
+--------------------+



In [22]:
dfwords=dflines.withColumn('words',split(col('value'),' '))\
 .withColumn('word',explode(col('words')))\
 .drop('value','words').groupBy('word').agg(count('word')\
 .alias('count')).orderBy('count',ascending=False).show()

+----------+-----+
|      word|count|
+----------+-----+
|       the|    9|
|        of|    7|
|     Lorem|    7|
|         a|    7|
|       and|    6|
|     Ipsum|    5|
|        is|    4|
|        it|    3|
|       has|    3|
|        It|    3|
|   content|    2|
|      that|    2|
|  versions|    2|
|publishing|    2|
|      page|    2|
|     their|    2|
|      when|    2|
|        by|    2|
|     dummy|    2|
|       use|    2|
+----------+-----+
only showing top 20 rows



## A partir du fichier Book. Compter les occurrences de mots à l’aide des méthodes issues des dataframes.

In [97]:
dflines=spark.read.text("./Exercices_Dataframe/Book")
dflines.show()

+--------------------+
|               value|
+--------------------+
|Self-Employment: ...|
|Achieving Financi...|
|       By Frank Kane|
|                    |
|                    |
|                    |
|Copyright � 2015 ...|
|All rights reserv...|
|                    |
|                    |
|            CONTENTS|
|          Disclaimer|
|             Preface|
|Part I: Making th...|
|  Overcoming Inertia|
|     Fear of Failure|
|Career Indoctrina...|
|The Carrot on a S...|
|      Ego Protection|
|Your Employer as ...|
+--------------------+
only showing top 20 rows



In [98]:
SplitDF = (dflines.select(split(dflines.value, '\s+').alias('split')))
SplitDF.show()

+--------------------+
|               split|
+--------------------+
|[Self-Employment:...|
|[Achieving, Finan...|
|   [By, Frank, Kane]|
|                  []|
|                  []|
|                  []|
|[Copyright, �, 20...|
|[All, rights, res...|
|                  []|
|                  []|
|          [CONTENTS]|
|        [Disclaimer]|
|           [Preface]|
|[Part, I:, Making...|
|[Overcoming, Iner...|
| [Fear, of, Failure]|
|[Career, Indoctri...|
|[The, Carrot, on,...|
|   [Ego, Protection]|
|[Your, Employer, ...|
+--------------------+
only showing top 20 rows



In [99]:
SingleDF = (SplitDF.select(explode(SplitDF.split).alias('word')))

SingleDFCount = SingleDF.count()

SingleDF.show()
print(SingleDFCount)

+----------------+
|            word|
+----------------+
|Self-Employment:|
|        Building|
|              an|
|        Internet|
|        Business|
|              of|
|             One|
|       Achieving|
|       Financial|
|             and|
|        Personal|
|         Freedom|
|         through|
|               a|
|       Lifestyle|
|      Technology|
|        Business|
|              By|
|           Frank|
|            Kane|
+----------------+
only showing top 20 rows



In [119]:
CleanDF = SingleDF.withColumn("clean", regexp_replace(SingleDF['word'], "[^A-Za-z0-9]", ""))
CleanDF = CleanDF.drop('word')
CleanDF.show()
# add '\W+ to regex

+--------------+
|         clean|
+--------------+
|SelfEmployment|
|      Building|
|            an|
|      Internet|
|      Business|
|            of|
|           One|
|     Achieving|
|     Financial|
|           and|
|      Personal|
|       Freedom|
|       through|
|             a|
|     Lifestyle|
|    Technology|
|      Business|
|            By|
|         Frank|
|          Kane|
+--------------+
only showing top 20 rows



In [121]:
dfwords=CleanDF.groupBy('clean').agg(count('clean')\
 .alias('count')).orderBy('count',ascending=False).show()

+--------+-----+
|   clean|count|
+--------+-----+
|      to| 1811|
|     you| 1361|
|    your| 1339|
|     the| 1177|
|       a| 1148|
|      of|  945|
|     and|  904|
|    that|  658|
|      in|  571|
|      is|  542|
|     for|  513|
|      on|  424|
|      it|  412|
|     are|  401|
|        |  369|
|      be|  358|
|business|  344|
|     can|  330|
|       I|  326|
|    have|  311|
+--------+-----+
only showing top 20 rows



# Exercice 5

## Calculer la somme des dépenses totales par consommateur à l’aide des méthodes issues des dataframes.

In [43]:
schema = StructType([\
    StructField("id", IntegerType(), True),\
    StructField("col2", IntegerType(), True),\
    StructField("depenses", FloatType(), True)])

In [44]:
df = (spark.read.format("csv").option("header","true").schema(schema).load("./Exercices_Dataframe/customer-orders.csv"))
df.show()

+---+----+--------+
| id|col2|depenses|
+---+----+--------+
| 35|5368|   65.89|
|  2|3391|   40.64|
| 47|6694|   14.98|
| 29| 680|   13.08|
| 91|8900|   24.59|
| 70|3959|   68.68|
| 85|1733|   28.53|
| 53|9900|   83.55|
| 14|1505|    4.32|
| 51|3378|    19.8|
| 42|6926|   57.77|
|  2|4424|   55.77|
| 79|9291|   33.17|
| 50|3901|   23.57|
| 20|6633|    6.49|
| 15|6148|   65.53|
| 44|8331|   99.19|
|  5|3505|   64.18|
| 48|5539|   32.42|
| 47|9900|   25.66|
+---+----+--------+
only showing top 20 rows



In [48]:
df.groupBy("id").sum("depenses").orderBy('id',ascending=True).show()

+---+------------------+
| id|     sum(depenses)|
+---+------------------+
|  0| 5524.950008839369|
|  1| 4958.599974133074|
|  2| 5994.589979887009|
|  3| 4659.629958629608|
|  4| 4815.050017774105|
|  5|4561.0700044333935|
|  6| 5397.880012750626|
|  7| 4755.070008277893|
|  8|5517.2399980425835|
|  9|  5322.64999294281|
| 10| 4819.699996152893|
| 11| 5152.289969373494|
| 12| 4664.589988231659|
| 13| 4367.619992315769|
| 14|4735.0300142765045|
| 15| 5413.510010659695|
| 16| 4979.059988319874|
| 17| 5032.680001735687|
| 18|4921.2700062170625|
| 19| 5059.429994106293|
+---+------------------+
only showing top 20 rows



# Exercice 6

## Classer les films par degré de popularité (Du plus populaire au moins populaire. Afficher les 10 premiers films les plus populaires.

In [56]:
dfdata=spark.read.("./Exercices_Dataframe/u.data")
dfdata.show()

+--------------------+
|               value|
+--------------------+
|196\t242\t3\t8812...|
|186\t302\t3\t8917...|
|22\t377\t1\t87888...|
|244\t51\t2\t88060...|
|166\t346\t1\t8863...|
|298\t474\t4\t8841...|
|115\t265\t2\t8811...|
|253\t465\t5\t8916...|
|305\t451\t3\t8863...|
| 6\t86\t3\t883603013|
|62\t257\t2\t87937...|
|286\t1014\t5\t879...|
|200\t222\t5\t8760...|
|210\t40\t3\t89103...|
|224\t29\t3\t88810...|
|303\t785\t3\t8794...|
|122\t387\t5\t8792...|
|194\t274\t2\t8795...|
|291\t1042\t4\t874...|
|234\t1184\t2\t892...|
+--------------------+
only showing top 20 rows



In [135]:
schema = StructType([\
    StructField("id_film", IntegerType(), True),\
    StructField("id_util", IntegerType(), True),\
    StructField("note", IntegerType(), True),\
    StructField("timestamp", IntegerType(), True)])

In [136]:
df = (spark.read.format("csv").option("header","false").option("sep","\t").schema(schema).load("./Exercices_Dataframe/u.data"))
df.show()

+-------+-------+----+---------+
|id_film|id_util|note|timestamp|
+-------+-------+----+---------+
|    196|    242|   3|881250949|
|    186|    302|   3|891717742|
|     22|    377|   1|878887116|
|    244|     51|   2|880606923|
|    166|    346|   1|886397596|
|    298|    474|   4|884182806|
|    115|    265|   2|881171488|
|    253|    465|   5|891628467|
|    305|    451|   3|886324817|
|      6|     86|   3|883603013|
|     62|    257|   2|879372434|
|    286|   1014|   5|879781125|
|    200|    222|   5|876042340|
|    210|     40|   3|891035994|
|    224|     29|   3|888104457|
|    303|    785|   3|879485318|
|    122|    387|   5|879270459|
|    194|    274|   2|879539794|
|    291|   1042|   4|874834944|
|    234|   1184|   2|892079237|
+-------+-------+----+---------+
only showing top 20 rows



In [137]:
df.groupBy("id_film").mean("note").orderBy("avg(note)",ascending=False).show(10)

+-------+-----------------+
|id_film|        avg(note)|
+-------+-----------------+
|    849|4.869565217391305|
|    688|4.833333333333333|
|    507|4.724137931034483|
|    628|4.703703703703703|
|    928|           4.6875|
|    118|4.661971830985915|
|    907|4.571428571428571|
|    686|4.563380281690141|
|    427|4.548387096774194|
|    565|4.542857142857143|
+-------+-----------------+
only showing top 10 rows



In [138]:
df.groupBy("id_film").agg({"note":"avg"}).orderBy("avg(note)",ascending=False).show(10)

+-------+-----------------+
|id_film|        avg(note)|
+-------+-----------------+
|    849|4.869565217391305|
|    688|4.833333333333333|
|    507|4.724137931034483|
|    628|4.703703703703703|
|    928|           4.6875|
|    118|4.661971830985915|
|    907|4.571428571428571|
|    686|4.563380281690141|
|    427|4.548387096774194|
|    565|4.542857142857143|
+-------+-----------------+
only showing top 10 rows

